In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rioxarray
import xarray as xr
from shapely.geometry import Polygon, box
import netCDF4
import os

c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
amm_pol = gpd.read_file(r"C:\Users\user\OneDrive\Desktop\new ap mort\Qasaba SHP\Amman.shp").set_crs(epsg=4326).to_crs(epsg=4326)
irb_pol = gpd.read_file(r"C:\Users\user\OneDrive\Desktop\new ap mort\Qasaba SHP\Qasabat Irbid.shp").to_crs(epsg=4326)
zar_pol = gpd.read_file(r"C:\Users\user\OneDrive\Desktop\new ap mort\Qasaba SHP\Qasabet Al Zarqa.shp").to_crs(epsg=4326)

In [3]:
amm_pol = amm_pol[irb_pol.columns]

In [4]:
folder = r"C:\Users\user\OneDrive\Desktop\new ap mort\NC data\NC files/"
files = os.listdir(folder)
files = [f for f in files if f.endswith('.grib')]
ncs = [
    xr.open_dataset(
        os.path.join(folder, file), 
        engine="cfgrib", 
        backend_kwargs={"indexpath": ""}
    ).coarsen(time=24, boundary='pad').mean()
    for file in files
]

In [5]:
for nc in ncs:
    nc['time'] = pd.DatetimeIndex(nc.indexes['time'].date)
    nc['valid_time'] = pd.DatetimeIndex(nc.indexes['time'].date)

In [6]:
def fine_grid_no_interpolation(cdf, regridding_factor, epsilon = 1e-9):

    """
    cdf: NetCDF4 dataset read by xarray.

    regridding_factor: the factor that we want to resample our dataset by; it has to be a multiplier of the original dimension,
    Eg. using a factor of 5 for 0.25 ERA5 will resample the data into 0.05 degree dataset.

    epsilon: a value to avoid floating point error when defining the new dimensions.

    This function resamples the dataset into finer dimension without interpolation, using Kroniker Product.

    Returns xarray dataset with the new spatial dimensions.
    """

    lons = cdf.variables[[name for name in cdf.coords if 'lon' in name.lower()][0]][:]
    lats = cdf.variables[[name for name in cdf.coords if 'lat' in name.lower()][0]][:]
    var = cdf.variables[list(cdf.data_vars.keys())[0]][:]
    dif_lon = lons[1] - lons[0]
    dif_lat = lats[0] - lats[1]

    lats_fine = np.float64(np.round(np.arange(lats[0] + (dif_lat/2) - (dif_lat/regridding_factor)/2,
                               lats[-1]- (dif_lat/2) + (dif_lat/regridding_factor)/2,
                               -dif_lat/regridding_factor+epsilon),4))

    lons_fine = np.float64(np.round(np.arange(lons[0] - (dif_lon/2) + (dif_lon/regridding_factor)/2,
                               lons[-1]+ (dif_lon/2) - (dif_lon/regridding_factor)/2,
                               dif_lon/regridding_factor-epsilon),4))

    t = cdf[[name for name in cdf.coords if 'time' in name.lower()][0]]
    times = pd.DatetimeIndex(t.to_pandas())

    var_kron = np.kron(var, np.ones((regridding_factor,regridding_factor)))

    var_name = list(cdf.data_vars.keys())[0]

    # Create a DataArray for the variable and then convert to a Dataset
    data_array = xr.DataArray(
        var_kron,
        coords=[times, lats_fine, lons_fine],
        dims=["time", "latitude", "longitude"],
        name=var_name
    )
    data_set = data_array.to_dataset()


    return data_set

In [7]:
def extracting_and_averaging_polygon(poly, nc_file, regridding_factor, epsilon = 1e-9):
  poly_4326 = poly.to_crs('epsg:4326')
  x = poly_4326.centroid.x.values[0]
  y = poly_4326.centroid.y.values[0]
  lons = nc_file.variables[[name for name in nc_file.coords if 'lon' in name.lower()][0]][:]
  lats = nc_file.variables[[name for name in nc_file.coords if 'lat' in name.lower()][0]][:]
  dif_lon = lons[1] - lons[0]
  dif_lat = lats[0] - lats[1]

  margin_lon = dif_lon/2 * 1.5
  margin_lat = dif_lat/2 * 1.5

  bounds = poly_4326['geometry'][0].bounds  # Get bounds in EPSG:4326
  maxi = max(max(abs(x - (bounds[0] - margin_lon)), abs(x - (bounds[2] + margin_lon))),
           max(abs(y - (bounds[1] - margin_lat)), abs(y - (bounds[3] + margin_lat))))

  rectangle_polygon_geometry = box(x - maxi - margin_lon,
                                 y - maxi - margin_lat,
                                 x + maxi + margin_lon,
                                 y + maxi + margin_lat)

  # Create the GeoDataFrame
  polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[rectangle_polygon_geometry])
  nc_file.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
  nc_file.rio.write_crs("epsg:4326", inplace=True)
  clipped = nc_file.rio.clip(polygon.geometry,polygon.crs)
  del lons, lats

  # Rename dimensions to match expected names in fine_grid_no_interpolation
  # Assuming the time dimension is named 'valid_time' in the input nc_file (t2m)
  original_time_dim = [name for name in clipped.dims if 'time' in name.lower()][0]
  clipped = clipped.rename({original_time_dim: 'time'})


  data_set = fine_grid_no_interpolation(clipped, regridding_factor, epsilon)
  data_set.rio.set_spatial_dims(x_dim=[name for name in data_set.coords if 'lon' in name.lower()][0],
                                y_dim=[name for name in data_set.coords if 'lat' in name.lower()][0],
                                inplace=True)
  data_set.rio.write_crs("epsg:4326", inplace=True)

  c = data_set.rio.clip(poly.geometry,polygon.crs)

  # Convert to DataFrame and inspect before groupby
  df_before_groupby = c.to_dataframe().dropna().reset_index()


  avg_df = df_before_groupby.groupby('time')[list(c.data_vars.keys())[0]].mean()
  return avg_df

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = []
for nc in ncs:
    df.append(extracting_and_averaging_polygon(amm_pol, nc, regridding_factor=25))
amm_df = pd.concat(df, axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = poly_4326.centroid.x.values[0]
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = poly_4326.centroid.y.values[0]
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:32: UserWarning: rename 'time' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  clipped = clipped.rename({original_time_dim: 'time'})
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely inco

In [9]:
df = []
for nc in ncs:
    df.append(extracting_and_averaging_polygon(irb_pol, nc, regridding_factor=25))
irb_df = pd.concat(df, axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = poly_4326.centroid.x.values[0]
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = poly_4326.centroid.y.values[0]
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:32: UserWarning: rename 'time' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  clipped = clipped.rename({original_time_dim: 'time'})
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely inco

In [10]:
df = []
for nc in ncs:
    df.append(extracting_and_averaging_polygon(zar_pol, nc, regridding_factor=10))
zar_df = pd.concat(df, axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = poly_4326.centroid.x.values[0]
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = poly_4326.centroid.y.values[0]
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:32: UserWarning: rename 'time' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  clipped = clipped.rename({original_time_dim: 'time'})
C:\Users\user\AppData\Local\Temp\ipykernel_748\420682152.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely inco

In [11]:
names = ['Amman', 'Irbid', 'Zarqa']
dfs = [amm_df, irb_df, zar_df]

In [12]:
for df in dfs:
    df['t2m'] = np.round(df['t2m']-273.15, 2)
    df['d2m'] = np.round(df['d2m']-273.15, 2)
    df['sp'] = np.round(df['sp']/1000, 4)  # Convert pressure from Pa to kPa

In [13]:
def calculate_relative_humidity(T, Dp):
    """
    Calculate relative humidity (RH) given temperature (T) and dew point (Dp).
    T and Dp should be in degrees Celsius.
    Returns RH in percentage.
    """
    numerator = np.exp((17.625 * Dp) / (243.04 + Dp))
    denominator = np.exp((17.625 * T) / (243.04 + T))
    RH = 100 * (numerator / denominator)
    return RH

In [14]:
for df in dfs:
    df['RH'] = np.round(calculate_relative_humidity(df['t2m'], df['d2m']), 2)
    df[df['RH'] <= 0] = 0  # Ensure RH is not negative
    df[df['RH'] >= 100] = 100  # Ensure RH does not exceed 100%

In [22]:
for i in range(len(dfs)):
    dfs[i].to_excel(r"C:\Users\user\OneDrive\Desktop\new ap mort\NC data\Excel/" + names[i] + ".xlsx", index=True, header=True)